# Checking the symplecticity of a method

First, we need to import the packages that will be used in the following code.

In [1]:
import Pkg; Pkg.add("BSeries")
import Pkg; Pkg.add("RootedTrees")
import Pkg; Pkg.add("Latexify")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using BSeries
using RootedTrees
using Latexify

## The Interesting Part
Now we can start to define the functions needed to check the symplecticity of a method.

The ideas mainly come from Sundklakk's master thesis and are only changed a little to translate it from python to julia.

First, we need a function that checks if the condition is satisfied for pair of trees of a specific order.

In [4]:
function SatisfiedForTreesOrder(condition, b, order)
    for order1 in range(1, order - 1)
        order2 = order - order1
        for tree1 in map(copy, RootedTreeIterator(order1))
            for tree2 in map(copy, RootedTreeIterator(order2))
                if condition(b, tree1, tree2) != true 
                    return false
                end
            end
        end
    end
    return true
end

SatisfiedForTreesOrder (generic function with 1 method)

Now we define the condition needed for the symplecticity.

In [6]:
function SymplecticityCondition(b::TruncatedBSeries, u, v)
    return b[u∘v] + b[v∘u] == b[u] * b[v]
end

SymplecticityCondition (generic function with 1 method)

Finally, we can define the function that returns the symplecticity order of a method.

In [7]:
function Symplecticity(b::TruncatedBSeries, max_order::Int64)
    order = 1
    
    if b[rootedtree(Int64[])] != 1
        return 0
        
    else
            
        for i in 1:max_order
            if in(rootedtree(collect(1:i)), keys(b))
                if SatisfiedForTreesOrder(SymplecticityCondition, b, i) == true                        
                    order = i
                else 
                    break 
                end
            else
                order = (order, "ATTENTION: max_order higher than order of b. This can affect the symplecticity order. Try again with order of b higher or equal than max_order.")
                break
            end
        end
    end
    return order
end

Symplecticity (generic function with 1 method)

# Example
## Implicit Midpoint Method
A good example is the Gauß Method with 2 stages (Implicit Midpoint).
Since the method is not only pseudo-symplectic but indeed symplectic (means we are having a pseudo-symplecticity order $\infty$, we can choose the max oder as high as we want to since the symplecticity order returned by the function will always be the same as the max oder. 

In [8]:
A = [1//2;;]
b = [1]
c = [1//2]
rk_implicitmidpoint = RungeKuttaMethod(A,b)
bseries_implicitmidpoint = bseries(rk_implicitmidpoint, 3)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 5 entries:
  RootedTree{Int64}: Int64[]   => 1//1
  RootedTree{Int64}: [1]       => 1//1
  RootedTree{Int64}: [1, 2]    => 1//2
  RootedTree{Int64}: [1, 2, 3] => 1//4
  RootedTree{Int64}: [1, 2, 2] => 1//4

In [9]:
Symplecticity(bseries_implicitmidpoint, 6)

(3, "ATTENTION: max_order higher than order of b. This can affect the symplecticity order. Try again with order of b higher or equal than max_order.")

## Any other explicit method

In [10]:
k = 3
A = [0 0 0;
(8*k-3)//(8k-4) 0 0;
(16k^2-8*k+1)//(2*k*(8k-3)) (2k-1)//(2*k*(8k-3)) 0]
b = [(3k-1)//(8k-3), (-2*(2k-1)^2)//(8k-3), k]
c = [0, (8k-3)//(8k-4), 1]
rkh = RungeKuttaMethod(A,b)
bser = bseries(rkh, 5)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 1//1
  RootedTree{Int64}: [1]             => 1//1
  RootedTree{Int64}: [1, 2]          => 1//2
  RootedTree{Int64}: [1, 2, 3]       => 1//8
  RootedTree{Int64}: [1, 2, 2]       => 3//8
  RootedTree{Int64}: [1, 2, 3, 4]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 3]    => 21//160
  RootedTree{Int64}: [1, 2, 3, 2]    => 1//8
  RootedTree{Int64}: [1, 2, 2, 2]    => 39//160
  RootedTree{Int64}: [1, 2, 3, 4, 5] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 4] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 441//3200
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 21//160
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 1//192
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 1//8
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 339//3200